# Using the MANN Package to train a Fully Connected Neural Network

In this notebook, the MANN package will be used to train pruned fully connected neural networks.  We will train two single-task networks on two separate tasks and one multitask network which performs both tasks.

In [1]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import mann

# Load the make_classification function from scikit-learn
from sklearn.datasets import make_classification

In [2]:
# We will use two separate generated datasets
x1, y1 = make_classification(
    n_samples = 10000,
    n_features = 10,
    n_informative = 8,
    n_classes = 2,
    n_clusters_per_class = 1
)

x2, y2 = make_classification(
    n_samples = 10000,
    n_features = 20,
    n_informative = 13,
    n_classes = 10,
    n_clusters_per_class = 1
)

# Flatten the outputs for simplicity
y1 = y1.reshape(-1, 1)
y2 = y2.reshape(-1, 1)

# Create a callback to stop training early
callback = tf.keras.callbacks.EarlyStopping(min_delta = 0.01, patience = 3, restore_best_weights = True)

## Create the first model

This first model is a fully connected model which will perform the first task. It will be pruned utilizing the MANN package so that most of its weights are 0.

In [3]:
# After data generation, create the single-task model using the TensorFlow Keras Functional API
input_layer = tf.keras.Input(x1.shape[-1])

# Instead of using keras Dense Layers, use MANN MaskedDense Layers
x = mann.layers.MaskedDense(
    100,
    activation = 'relu'
)(input_layer)

for _ in range(5):
    x = mann.layers.MaskedDense(
        100,
        activation = 'relu'
    )(x)

# Create the output layer as another MANN MaskedDense Layer
output_layer = mann.layers.MaskedDense(1, activation = 'sigmoid')(x)

# Create the model
model = tf.keras.Model(input_layer, output_layer)

Metal device set to: Apple M1


2021-11-09 08:24:33.181999: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-09 08:24:33.182119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Compile the model for training and masking
model.compile(
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask (prune) the model using the MANN package
model = mann.utils.mask_model(
    model = model,              # The model to be pruned
    percentile = 90,            # The percentile to be masked, for example, if the value is 90, then 90% of weights will be masked
    method = 'gradients',       # The method to use to mask, either 'gradients' or 'magnitude'
    exclusive = True,           # Whether weight locations must be exclusive to each task
    x = x1[:2000],              # The input data
    y = y1[:2000]               # The expected outputs
)

# Recompile the model
model.compile(
    loss = 'binary_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

In [5]:
# To show how the layers of the model have been pruned, output the kernel of the first MaskedDense Layer
model.layers[1].get_weights()[0]

array([[ 3.90325785e-02, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -2.18045320e-02,  0.00000000e+00, -0.00000000e+00,
        -1.02299631e-01,  0.00000000e+00, -0.00000000e+00,
         0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00, -0.00000000e+00, -6.97451178e-03,
        -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  1.79088693e-02,
        -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00, -7.33782649e-0

In [6]:
# Fit the model on the first dataset
model.fit(x1, y1, batch_size = 128, epochs = 100, validation_split = 0.2, callbacks = [callback])
print(f'First Model Accuracy: {((model.predict(x1)>= 0.5).astype(int).flatten() == y1.flatten()).sum()/y1.shape[0]}')

2021-11-09 08:26:35.223779: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-09 08:26:35.224169: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-09 08:26:35.420391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/100
63/63 [==============================] - 2s 10ms/step - loss: 0.6903 - accuracy: 0.5299 - val_loss: 0.6650 - val_accuracy: 0.7810
Epoch 2/100
 1/63 [..............................] - ETA: 0s - loss: 0.6668 - accuracy: 0.7422

2021-11-09 08:26:36.874633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 0s 6ms/step - loss: 0.4524 - accuracy: 0.8561 - val_loss: 0.2396 - val_accuracy: 0.9075
Epoch 3/100
63/63 [==============================] - 0s 6ms/step - loss: 0.2254 - accuracy: 0.9100 - val_loss: 0.1915 - val_accuracy: 0.9285
Epoch 4/100
63/63 [==============================] - 0s 6ms/step - loss: 0.1987 - accuracy: 0.9203 - val_loss: 0.1747 - val_accuracy: 0.9310
Epoch 5/100
63/63 [==============================] - 0s 6ms/step - loss: 0.1819 - accuracy: 0.9298 - val_loss: 0.1584 - val_accuracy: 0.9395
Epoch 6/100
63/63 [==============================] - 0s 6ms/step - loss: 0.1599 - accuracy: 0.9389 - val_loss: 0.1336 - val_accuracy: 0.9535
Epoch 7/100
63/63 [==============================] - 0s 6ms/step - loss: 0.1347 - accuracy: 0.9518 - val_loss: 0.1042 - val_accuracy: 0.9685
Epoch 8/100
63/63 [==============================] - 0s 6ms/step - loss: 0.1110 - accuracy: 0.9644 - val_loss: 0.0865 - val_accuracy: 0.9745
Epoch 9/100
63/63 [======

2021-11-09 08:26:42.428416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


First Model Accuracy: 0.9838


## Create the second model

This second model is a fully connected model which will perform the second task. It will be pruned utilizing the MANN package so that most of its weights are 0.

In [7]:
# Create the second model
input_layer = tf.keras.Input(x2.shape[-1])

# Instead of using keras Dense Layers, use MANN MaskedDense Layers
x = mann.layers.MaskedDense(
    100,
    activation = 'relu'
)(input_layer)

for _ in range(5):
    x = mann.layers.MaskedDense(
        100,
        activation = 'relu'
    )(x)

# Create the output layer as another MANN MaskedDense Layer
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

# Create the model
model = tf.keras.Model(input_layer, output_layer)

In [8]:
# Repeat the pruning process for the second model
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

model = mann.utils.mask_model(
    model = model,
    percentile = 90,
    method = 'gradients',
    exclusive = True,
    x = x2[:2000],
    y = y2.reshape(-1, 1)[:2000]
)

model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

model.fit(x2, y2, epochs = 100, batch_size = 128, validation_split = 0.2, callbacks = [callback])

print(f'Second Model Accuracy: {(model.predict(x2).argmax(axis = 1) == y2.flatten()).astype(int).sum()/y2.shape[0]}')

Epoch 1/100


2021-11-09 08:29:35.230061: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


62/63 [============================>.] - ETA: 0s - loss: 2.2996 - accuracy: 0.1070

2021-11-09 08:29:36.506774: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 2s 13ms/step - loss: 2.2993 - accuracy: 0.1073 - val_loss: 2.2719 - val_accuracy: 0.1410
Epoch 2/100
63/63 [==============================] - 0s 7ms/step - loss: 2.2235 - accuracy: 0.1600 - val_loss: 2.1449 - val_accuracy: 0.1795
Epoch 3/100
63/63 [==============================] - 0s 6ms/step - loss: 2.1064 - accuracy: 0.1773 - val_loss: 2.0628 - val_accuracy: 0.1975
Epoch 4/100
63/63 [==============================] - 0s 7ms/step - loss: 2.0460 - accuracy: 0.1931 - val_loss: 2.0103 - val_accuracy: 0.2240
Epoch 5/100
63/63 [==============================] - 0s 7ms/step - loss: 1.9907 - accuracy: 0.2244 - val_loss: 1.9565 - val_accuracy: 0.2540
Epoch 6/100
63/63 [==============================] - 0s 7ms/step - loss: 1.9345 - accuracy: 0.2633 - val_loss: 1.9224 - val_accuracy: 0.2625
Epoch 7/100
63/63 [==============================] - 0s 7ms/step - loss: 1.8932 - accuracy: 0.2869 - val_loss: 1.8924 - val_accuracy: 0.2655
Epoch 8/100
63/63 [=====

2021-11-09 08:30:00.912786: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Second Model Accuracy: 0.7609


## Create the MANN

The third and final model we create here will be a multitask model (MANN) which performs both tasks.

In [9]:
# Train a Multitask Model

input1 = tf.keras.layers.Input(x1.shape[-1])
input2 = tf.keras.layers.Input(x2.shape[-1])
dense1 = mann.layers.MaskedDense(100, activation = 'relu')(input1)
dense2 = mann.layers.MaskedDense(100, activation = 'relu')(input2)
x = mann.layers.MultiMaskedDense(100, activation = 'relu')([dense1, dense2])
for _ in range(4):
    x = mann.layers.MultiMaskedDense(100, activation = 'relu')(x)
sel1 = mann.layers.SelectorLayer(0)(x)
sel2 = mann.layers.SelectorLayer(1)(x)
output1 = mann.layers.MaskedDense(1, activation = 'sigmoid')(sel1)
output2 = mann.layers.MaskedDense(10, activation = 'sigmoid')(sel2)

model =  tf.keras.Model([input1, input2], [output1, output2])
model.compile(
    loss = ['binary_crossentropy', 'sparse_categorical_crossentropy'],
    metrics = ['accuracy'],
    optimizer = 'adam'
)
model = mann.utils.mask_model(
    model,
    90,
    method = 'gradients',
    exclusive = True,
    x = [x1[:2000], x2[:2000]],
    y = [y1.reshape(-1, 1)[:2000], y2.reshape(-1, 1)[:2000]]
)
model.compile(
    loss = ['binary_crossentropy', 'sparse_categorical_crossentropy'],
    metrics = ['accuracy'],
    optimizer = 'adam'
)

model.fit([x1, x2], [y1, y2], epochs = 100, batch_size = 128, callbacks = [callback], validation_split = 0.2)
p1, p2 = model.predict([x1, x2])
p1 = (p1 >= 0.5).astype(int)
p2 = p2.argmax(axis = 1)

Epoch 1/100


2021-11-09 08:30:02.951589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 2.9672 - masked_dense_16_loss: 0.6649 - masked_dense_17_loss: 2.3023 - masked_dense_16_accuracy: 0.5364 - masked_dense_17_accuracy: 0.1153

2021-11-09 08:30:05.056296: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 3s 27ms/step - loss: 2.9672 - masked_dense_16_loss: 0.6649 - masked_dense_17_loss: 2.3023 - masked_dense_16_accuracy: 0.5364 - masked_dense_17_accuracy: 0.1153 - val_loss: 2.8330 - val_masked_dense_16_loss: 0.5336 - val_masked_dense_17_loss: 2.2994 - val_masked_dense_16_accuracy: 0.7080 - val_masked_dense_17_accuracy: 0.1515
Epoch 2/100
63/63 [==============================] - 1s 16ms/step - loss: 2.7088 - masked_dense_16_loss: 0.4750 - masked_dense_17_loss: 2.2338 - masked_dense_16_accuracy: 0.8455 - masked_dense_17_accuracy: 0.1543 - val_loss: 2.6430 - val_masked_dense_16_loss: 0.4385 - val_masked_dense_17_loss: 2.2046 - val_masked_dense_16_accuracy: 0.8975 - val_masked_dense_17_accuracy: 0.1740
Epoch 3/100
63/63 [==============================] - 1s 16ms/step - loss: 2.5521 - masked_dense_16_loss: 0.4144 - masked_dense_17_loss: 2.1377 - masked_dense_16_accuracy: 0.9056 - masked_dense_17_accuracy: 0.1779 - val_loss: 2.4594 - val_masked_dense_1

Epoch 23/100
63/63 [==============================] - 1s 15ms/step - loss: 1.1522 - masked_dense_16_loss: 0.0536 - masked_dense_17_loss: 1.0986 - masked_dense_16_accuracy: 0.9875 - masked_dense_17_accuracy: 0.6391 - val_loss: 1.1659 - val_masked_dense_16_loss: 0.0476 - val_masked_dense_17_loss: 1.1183 - val_masked_dense_16_accuracy: 0.9885 - val_masked_dense_17_accuracy: 0.6290
Epoch 24/100
63/63 [==============================] - 1s 15ms/step - loss: 1.1225 - masked_dense_16_loss: 0.0521 - masked_dense_17_loss: 1.0704 - masked_dense_16_accuracy: 0.9879 - masked_dense_17_accuracy: 0.6488 - val_loss: 1.1449 - val_masked_dense_16_loss: 0.0453 - val_masked_dense_17_loss: 1.0996 - val_masked_dense_16_accuracy: 0.9890 - val_masked_dense_17_accuracy: 0.6330
Epoch 25/100
63/63 [==============================] - 1s 15ms/step - loss: 1.0979 - masked_dense_16_loss: 0.0517 - masked_dense_17_loss: 1.0462 - masked_dense_16_accuracy: 0.9879 - masked_dense_17_accuracy: 0.6538 - val_loss: 1.1154 - val

63/63 [==============================] - 1s 15ms/step - loss: 0.8155 - masked_dense_16_loss: 0.0423 - masked_dense_17_loss: 0.7732 - masked_dense_16_accuracy: 0.9906 - masked_dense_17_accuracy: 0.7480 - val_loss: 0.8745 - val_masked_dense_16_loss: 0.0387 - val_masked_dense_17_loss: 0.8358 - val_masked_dense_16_accuracy: 0.9915 - val_masked_dense_17_accuracy: 0.7240
Epoch 45/100
63/63 [==============================] - 1s 15ms/step - loss: 0.8128 - masked_dense_16_loss: 0.0418 - masked_dense_17_loss: 0.7710 - masked_dense_16_accuracy: 0.9904 - masked_dense_17_accuracy: 0.7465 - val_loss: 0.8796 - val_masked_dense_16_loss: 0.0393 - val_masked_dense_17_loss: 0.8404 - val_masked_dense_16_accuracy: 0.9910 - val_masked_dense_17_accuracy: 0.7200
Epoch 46/100
63/63 [==============================] - 1s 15ms/step - loss: 0.8008 - masked_dense_16_loss: 0.0409 - masked_dense_17_loss: 0.7599 - masked_dense_16_accuracy: 0.9913 - masked_dense_17_accuracy: 0.7548 - val_loss: 0.8633 - val_masked_dense

2021-11-09 08:30:59.567906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


# Predict Using the MANN

Now that the MANN model has been trained, we can use it to get predictions just as we would a traditional model. In this case, a list of predictions are returned, with each index corresponding to the task.

In [10]:
print(f'Multitask Task 1 Accuracy: {(p1.flatten() == y1.flatten()).sum()/y1.flatten().shape[0]}')
print(f'Multitask Task 2 Accuracy: {(p2.flatten() == y2.flatten()).sum()/y2.flatten().shape[0]}')

Multitask Task 1 Accuracy: 0.9884
Multitask Task 2 Accuracy: 0.7721
